In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# Read data from csv
iris = pd.read_csv("data/Iris.csv")
print(iris.head())

# Plot the various combinations of 2D graph
#g = sns.pairplot(iris.drop("Id", axis=1), hue="Species")

# Replace the species with 0, 1 or 2 as appropriate
iris['Species'].replace(['Iris-setosa', 'Iris-virginica', 'Iris-versicolor'], [0, 1, 2], inplace=True)

# Get labels, flatten and encode to one-hot
columns = ['Species']
labels = pd.DataFrame(iris, columns=columns).to_numpy()
labels = labels.flatten()
labels = np.eye(np.max(labels) + 1)[labels]

# Get Features
columns = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
features = pd.DataFrame(iris, columns=columns).to_numpy()

# Split data to training and test data, 2/3 for training and 1/3 for testing
train_x, test_x, train_y, test_y = train_test_split(features, labels, test_size=0.33)

train and test

In [ ]:
# Sigmoid and its derivative

def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_deriv(x):
    return sigmoid(x) * (1 - sigmoid(x))

# Learning rate
learning_rate = 0.01

# Number of training epochs
num_epochs = 200

# Network architecture parameters
num_features = len(train_x[0])
num_classes = len(train_y[0])
num_hidden_nodes = 8

# Initialise weights in the range -1 to 1
# np.random.seed(1)
# Hidden layer weights with shape = number of input features x number of hidden nodes
hidden_weights = np.random.uniform(-1, 1, size=(num_features, num_hidden_nodes))
hidden_bias = np.random.uniform(-1, 1, size=(1, num_hidden_nodes))
# Output layer weights with shape = number of hidden nodes x number of output classes
output_weights = np.random.uniform(-1, 1, size=(num_hidden_nodes, num_classes))
output_bias = np.random.uniform(-1, 1, size=(1, num_classes))

# For recording error and accuracy - for graph later
training_errors = []
testing_errors = []
training_accuracies = []
testing_accuracies = []

# Train for number of epochs
for epoch in range(num_epochs):

    # Forward pass
    input_layer = train_x
    # sigmoid( (W * X) + b)
    hidden_layer = sigmoid(np.dot(input_layer, hidden_weights) + hidden_bias)
    output_layer = sigmoid(np.dot(hidden_layer, output_weights) + output_bias)

    """ Backpropagation using gradient descent """ 
    # Calculate output layer error
    output_layer_error = (train_y - output_layer)

    # Calculate output layer derivative Note: that we just need this layers error for the bias
    output_layer_delta = output_layer_error * sigmoid_deriv(output_layer)
    output_bias_delta = np.sum(output_layer_error, axis=0)
    
    # Calculate hidden layer error (from the output layers weights and derivative
    hidden_layer_error = output_layer_delta.dot(output_weights.T)
    # Calculate hidden layer derivative Note: that we just need this layers error for the bias
    hidden_layer_delta = hidden_layer_error * sigmoid_deriv(hidden_layer)
    hidden_bias_delta = np.sum(hidden_layer_error, axis=0)

    # Update the weights (learning rate X layers input X layers derivative)
    output_weights += learning_rate * hidden_layer.T.dot(output_layer_delta)
    output_bias += learning_rate * output_bias_delta
    
    hidden_weights += learning_rate * input_layer.T.dot(hidden_layer_delta)
    hidden_bias += learning_rate * hidden_bias_delta

    # Every 100 epochs record error and accuracy during training
    if (epoch % 10) == 0:
        
        # Mean squared error over all errors this epoch
        error = np.square(output_layer_error).mean() 
        training_errors.append(error)

        accuracy_count = 0
        for i in range(len(output_layer)):
          
            # Get the prediction i.e. the output with the highest value
            prediction = np.argmax(output_layer[i])
            # Get the actual label
            actual_label = np.argmax(train_y[i])
            
            # If they match the prediction was correct
            if prediction == actual_label:
                accuracy_count += 1
        accuracy = (len(train_x) / 100) * accuracy_count
        training_accuracies.append(accuracy)
               
        # Forward pass
        test_hidden = sigmoid(np.dot(test_x, hidden_weights) + hidden_bias)
        test_output = sigmoid(np.dot(test_hidden, output_weights) + output_bias)
        test_output_error = test_y - test_output
        # Mean squared error over all errors
        test_error = np.square(test_output_error).mean()
        testing_errors.append(test_error)

        test_accuracy_count = 0
        for j in range(len(test_output)):

            # Get the prediction i.e. the output with the highest value
            test_prediction = np.argmax(test_output[j])
            # Get the actual label
            actual_label = np.argmax(test_y[j])

            # If they match the prediction was correct
            if test_prediction == actual_label:
                test_accuracy_count += 1

        test_accuracy = (100 / len(test_x)) * test_accuracy_count
        testing_accuracies.append(test_accuracy)
        
           
        
        print("Epoch: " + str(epoch) +
              " Error: " + str(round(error, 5)) +
              " Accuracy: " + str(accuracy) + "%" +
              " Test Error: " + str(round(test_error, 5)) +
              " Accuracy: " + str(test_accuracy) + "%")